In [15]:
# Подсчет accuracy и WER для модели Gemma-3n-E4B
# 1. Подсчитаем для текстовой генерации, когда на входе были текстовые запросы,
# на выходе текстовый ответ модели

import json
import re

def normalize_text(text):
    # Приводит текст к нижнему регистру и чистит от пробелов
    return text.lower().strip()

def remove_end_of_turn(text):
    # Удаляет <end_of_turn> из текста"""
    return re.sub(r'\s*<end_of_turn>\s*', '', text)

def parse_json_file(filename):
    # Считывает JSON-файл и возвращает данные"""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def has_tool_call(text):
    #Проверяет, содержит ли текст тег <tool_call>
    return '<tool_call>' in text

In [17]:
#jsonfile = 'gemma_audio_va_results2.json'
jsonfile = 'gemma_audio_va_results3fn.json'
data = parse_json_file(jsonfile)

In [3]:
for item in data[15:20]:
    print(item)

{'request': 'Launch Louis Armstrong', 'ground_truth': '<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title></title><genre></genre><album></album></arguments></tool_call>', 'real_response': '<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title></title><genre></genre><album></album></arguments></tool_call><end_of_turn>'}
{'request': 'I want to hear Jimi Hendrix', 'ground_truth': '<tool_call><name>audioplay.play_request</name><arguments><artist>Jimi Hendrix</artist><title></title><genre></genre><album></album></arguments></tool_call>', 'real_response': '<tool_call><name>audioplay.play_request</name><arguments><artist>Jimi Hendrix</artist><title></title><genre></genre><album></album></arguments></tool_call><end_of_turn>'}
{'request': 'Play something from Bad Bunny', 'ground_truth': '<tool_call><name>audioplay.play_request</name><arguments><artist>Bad Bunny</artist><title></title><genre></genre><album></a

In [18]:
def calculate_accuracy1(data):
    # Вычисляет accuracy как точное совпадение после нормализации ground_truth и real_response
    correct_count = 0
    total = len(data)
    
    for item in data:
        ground_truth = normalize_text(item['ground_truth'])
        real_response = normalize_text(remove_end_of_turn(item['real_response']))
        
        if ground_truth == real_response:
            correct_count += 1
    
    return correct_count / total if total > 0 else 0

In [19]:
def calculate_accuracy2(data):
    # Вычисляет accuracy, считая правильным ответом любой ответ без <tool_call>, если
    # его нет в ground_truth
    
    correct_count = 0
    total = len(data)
    
    for item in data:
        ground_truth = normalize_text(item['ground_truth'])
        real_response = normalize_text(remove_end_of_turn(item['real_response']))
        
        # Проверяем, начинается ли ground_truth с <tool_call>
        ground_truth_starts_with_toolcall = ground_truth.startswith('<tool_call>')
        real_response_starts_with_toolcall = real_response.startswith('<tool_call>')
        
        # Правило accuracy2: если оба не начинаются с <tool_call>, считаем правильным
        if not ground_truth_starts_with_toolcall and not real_response_starts_with_toolcall:
            correct_count += 1
        # Иначе сравниваем как в accuracy1
        elif ground_truth == real_response:
            correct_count += 1
    
    return correct_count / total if total > 0 else 0

In [20]:
def calculate_false_alarm_rate(data):
    # Вычисляет False Alarm Rate (FAR) - когда в запросе не было вызова, а в ответе есть tool_call
    false_alarms = 0
    ground_truth_without_toolcall = 0
    
    for item in data:
        ground_truth = normalize_text(item['ground_truth'])
        real_response = normalize_text(remove_end_of_turn(item['real_response']))
        
        # Проверяем наличие <tool_call> в ground_truth и real_response
        gt_has_toolcall = has_tool_call(ground_truth)
        rr_has_toolcall = has_tool_call(real_response)
        
        # Если в ground_truth нет tool_call, но в real_response есть
        if not gt_has_toolcall:
            ground_truth_without_toolcall += 1
            if rr_has_toolcall:
                false_alarms += 1
    
    # Вычисляем FAR (избегаем деления на ноль)
    far = false_alarms / ground_truth_without_toolcall if ground_truth_without_toolcall > 0 else 0
    
    return false_alarms, ground_truth_without_toolcall, far

def calculate_missed_detection_rate(data):
    # Missed Detection Rate (MDR).  В response нет tool_call
    missed_detections = 0
    ground_truth_with_toolcall = 0
    
    for item in data:
        ground_truth = normalize_text(item['ground_truth'])
        real_response = normalize_text(remove_end_of_turn(item['real_response']))
        
        # Проверяем наличие <tool_call> в ground_truth и real_response
        gt_has_toolcall = has_tool_call(ground_truth)
        rr_has_toolcall = has_tool_call(real_response)
        
        # Если в ground_truth есть tool_call, но в real_response нет
        if gt_has_toolcall:
            ground_truth_with_toolcall += 1
            if not rr_has_toolcall:
                missed_detections += 1
    
    # Вычисляем MDR (избегаем деления на ноль)
    mdr = missed_detections / ground_truth_with_toolcall if ground_truth_with_toolcall > 0 else 0
    
    return missed_detections, ground_truth_with_toolcall, mdr

In [21]:
print(f"accuracy1 = {calculate_accuracy1(data)}")
print(f"accuracy2 = {calculate_accuracy2(data)}")
print(f"FAR = {calculate_false_alarm_rate(data)}")
print(f"Missed Detection Rate (MDR) = {calculate_missed_detection_rate(data)}")

accuracy1 = 0.35306122448979593
accuracy2 = 0.3551020408163265
FAR = (77, 79, 0.9746835443037974)
Missed Detection Rate (MDR) = (1, 411, 0.0024330900243309003)


Результат такой, что практически полезный случай - это accuracy2 = 30%
При этом False Alarm Rate = 0  (то есть, когда ошибочно вызван инструмент)
Mission Detection Rate = 5% (то есть, когда инструмент вообще не вызван )

После небольшого файнтьюнинга:

accuracy1 = 0.35306122448979593
accuracy2 = 0.3551020408163265
FAR = (77, 79, 0.9746835443037974)
Missed Detection Rate (MDR) = (1, 411, 0.0024330900243309003)

Видно, что улучшилась точность. Особенно в тех случаях, когда модель ошибочно не вызывала tool_call. Но произошло это за счет резкого роста FAR. Которое стало почти 100%. То есть, модель теперь пытается вызвать tool_call в любом случае.  По этой же причине снизилась MDR

In [22]:
# Теперь проверим результат ASR
import csv

def parse_csv_file(filename):
    """Считывает CSV-файл и возвращает данные в формате списка словарей"""
    data = []
    
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            # Приводим имена полей к нижнему регистру для универсальности
            row_lower = {k.lower().strip(): v for k, v in row.items()}
            
            # Проверяем, что есть необходимые поля
            if 'assistant_text' in row_lower and 'asr_text' in row_lower:
                data.append({
                    'ground_truth': row_lower['assistant_text'],
                    'real_response': row_lower['asr_text']
                })
            else:
                print(f"Предупреждение: строка {reader.line_num} не содержит необходимых полей")
    
    return data

In [23]:
filename_csv = 'audioplayer_va_asr_fn.csv'
datacsv = parse_csv_file(filename_csv)

In [24]:
print(f"accuracy1 = {calculate_accuracy1(datacsv)}")
print(f"accuracy2 = {calculate_accuracy2(datacsv)}")
print(f"FAR = {calculate_false_alarm_rate(datacsv)}")
print(f"Missed Detection Rate (MDR) = {calculate_missed_detection_rate(datacsv)}")

accuracy1 = 0.09795918367346938
accuracy2 = 0.09795918367346938
FAR = (157, 158, 0.9936708860759493)
Missed Detection Rate (MDR) = (0, 822, 0.0)


Как видно, по сравнению с чисто текстовым распознаванием добавились ошибки при распознавании речи.

теперь acc1=3.4% (было 13%)<br/>
acc2 = 18% (было 30%)<br/>
FAR = 12% (было 0%)<br/>
MDR = 1.8% (было 5%) - улучшилось, но видимо за счет ухудшения FAR<br/>

После файнтьюнинга:

accuracy1 = 0.09795918367346938<br/>
accuracy2 = 0.09795918367346938<br/>
FAR = (157, 158, 0.9936708860759493)<br/>
Missed Detection Rate (MDR) = (0, 822, 0.0)<br/>

accuracy и MDR тоже стал лучше, почти 10%.  FAR катастрофически хуже. 